# House Prices: Advanced Regression Techniques

> Date:2018-10-29,Author:Ho Loong

1. **Define the Problem:** If data science, big data, machine learning, predictive analytics, business intelligence, or any other buzzword is the solution, then what is the problem? As the saying goes, don't put the cart before the horse. Problems before requirements, requirements before solutions, solutions before design, and design before technology. Too often we are quick to jump on the new shiny technology, tool, or algorithm before determining the actual problem we are trying to solve.
2. **Gather the Data:** John Naisbitt wrote in his 1984 (yes, 1984) book Megatrends, we are “drowning in data, yet staving for knowledge." So, chances are, the dataset(s) already exist somewhere, in some format. It may be external or internal, structured or unstructured, static or streamed, objective or subjective, etc. As the saying goes, you don't have to reinvent the wheel, you just have to know where to find it. In the next step, we will worry about transforming "dirty data" to "clean data."
3. **Prepare Data for Consumption:** This step is often referred to as data wrangling, a required process to turn “wild” data into “manageable” data. Data wrangling includes implementing data architectures for storage and processing, developing data governance standards for quality and control, data extraction (i.e. ETL and web scraping), and data cleaning to identify aberrant, missing, or outlier data points.
4. **Perform Exploratory Analysis:** Anybody who has ever worked with data knows, garbage-in, garbage-out (GIGO). Therefore, it is important to deploy descriptive and graphical statistics to look for potential problems, patterns, classifications, correlations and comparisons in the dataset. In addition, data categorization (i.e. qualitative vs quantitative) is also important to understand and select the correct hypothesis test or data model.
5. **Model Data:** Like descriptive and inferential statistics, data modeling can either summarize the data or predict future outcomes. Your dataset and expected results, will determine the algorithms available for use. It's important to remember, algorithms are tools and not magical wands or silver bullets. You must still be the master craft (wo)man that knows how-to select the right tool for the job. An analogy would be asking someone to hand you a Philip screwdriver, and they hand you a flathead screwdriver or worst a hammer. At best, it shows a complete lack of understanding. At worst, it makes completing the project impossible. The same is true in data modelling. The wrong model can lead to poor performance at best and the wrong conclusion (that’s used as actionable intelligence) at worst.
6. **Validate and Implement Data Model:** After you've trained your model based on a subset of your data, it's time to test your model. This helps ensure you haven't overfit your model or made it so specific to the selected subset, that it does not accurately fit another subset from the same dataset. In this step we determine if our [model overfit, generalize, or underfit our dataset](http://docs.aws.amazon.com/machine-learning/latest/dg/model-fit-underfitting-vs-overfitting.html).
7. **Optimize and Strategize:** This is the "bionic man" step, where you iterate back through the process to make it better...stronger...faster than it was before. As a data scientist, your strategy should be to outsource developer operations and application plumbing, so you have more time to focus on recommendations and design. Once you're able to package your ideas, this becomes your “currency exchange" rate.

# Data Read

1. 训练数据1460，测试数据1460。
2. 特征中大部分是枚举型，少部分是数值型和时间数据，需要mapping、convert，需要在full之后进行。
	1. 枚举型的值具有数值关系的（质量相关的），比如厨房质量：Ex>Gd>Ta>Fa>Po，那么mapping到数值上时这种关系要不要用数字体现。
	2. 其余枚举型是没有数值关系的，比如House Stype等。
3. 特征很多，70+，对应数据量却很少，需要feature Selection或者PCA。
4. (YearBuilt)关于建造年份、月份等，可以create一个房龄的feature。
5. 关于(YrSale,MoSale)销售年份、月份等，可以create一个至今销售时长的feature。
6. (YearRemodAdd)根据改建年份，create是否有改建的feature以及改建后的房龄的feature。
6. 部分数值型数据可能要cut处理来减少特征值。
7. 部分Feature的NA值通常是因为没有相关设施导致的，比如泳池面积，NA表示没有泳池，那么用0表示，而字符型的比如车库类型这种，NA用'None'表示代表没有车库的情况。
8. SaleType和SaleCondition看着对结果影响很大的样子。

# Step 1:Define Problem

[Kaggle:House Prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

* Supervise/Unsupervise:Supervise
* Classification/Regression:Regression
* Eval:RMSE
* Train Data:'./input/train.csv', len=1460
* Test Data:'./input/test.csv', len=1460

# Step 2:Gather the Data

Kaggle provide data.

# Step 3:Prepare Data for Consumption

## 3.1 Import Libs

### 3.11 Import Common Libs

In [1]:
import sys,os,time

import numpy as np
import pandas as pd

### 3.12 Import Model Libs

In [2]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBRegressor

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

#Configure Visualization Defaults
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

## 3.2 Meet and Greet Data

In [3]:
train_data = pd.read_csv('./input/train.csv')
test_data = pd.read_csv('./input/test.csv')

data_cleaner = [train_data, test_data]

print train_data.info()
print '-'*100
print train_data.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

### 3.21 The 4 C`s of Data Cleaning:Correcting, Completing, Creating and Conveting

In [4]:
print 'Train columns with null values:\n'
print train_data.isnull().sum()[train_data.isnull().sum()>0]
print "-"*100

print 'Test columns with null values:\n'
print test_data.isnull().sum()[test_data.isnull().sum()>0]
print "-"*100

Train columns with null values:

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64
----------------------------------------------------------------------------------------------------
Test columns with null values:

MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Fu

**缺失数据分析**

缺失通常有两个原因，一个是技术性缺失，一个是合理性缺失，通常技术性缺失需要通过平均值、其他相关字段等来补充，而合理性缺失通常用一个特殊值来表示缺失，即缺失也是一种取值，或者压根不补充都是可以的。

* LotFrontage:相连的街道的线脚，类型为数值型，暂无full方式，使用平均值填充。
* Alley:胡同类型，枚举型，null表示没有胡同，直接用'None'填充。
* MasVnrType:表层砌体，枚举型，null表示没有，直接用'None'填充。
* MasVnrArea:表层砌体面积，数值型，null表示没有，那么用0填充。
* BsmtQual:地下室高度的评价，意义枚举型，null表示没有，用'None'填充。
* BsmtCond:地下室一般情况的评价，意义枚举型，null表示没有，用'None'填充。
* BsmtExposure:地下室、水平墙，枚举型，null表示没有，用'None'填充。
* BsmtFinType1:地下室竣工面积评价，意义枚举型，null表示没有，用'None'填充。
* BsmtFinSF1:地下室竣工面积，数值型枚举，null表示没有，用0填充。
* BsmtFinType2:同BsmtFinType1。
* BsmtFinSF2:同BsmtFinSF1。
* BsmtUnfSF:未完工的地下室面积，数值型，null表示没有，用0填充。
* TotalBsmtSF:地下室总面积，数值型，null表示没有，用0填充。
* BsmtFullBath:地下室全浴室个数，数值型，null表示没有，用0填充。
* BsmtHalfBath:地下室半浴室个数，数值型，null表示没有，用0填充。
* Electrical:电力系统，枚举型，null用最多的那个值来填充。
* FireplaceQu:壁炉质量，意义枚举型，null表示没有，用'None'来填充。
* GarageType:车库位置，枚举型，null表示没有，用'None'填充。
* GarageYrBlt:车库修建年份，数值型，null表示没有，用0填充。
* GarageFinish:车库内饰完成度，枚举型，null表示没有车库，用'None'填充。
* GarageQual:车库质量评价，意义枚举型，null表示没有车库，用'None'填充。
* GarageCond:基本同上。
* GarageCars:车库车位数，数值型，null表示没有车库，用0填充。
* GarageArea:车库面积，数值型，null表示没有车库，用0填充。
* PoolQC:泳池质量，意义枚举型，null表示没有，用'None'填充。
* Fence:栅栏质量，意义枚举型，null表示没有，用'None'填充。
* MiscFeature:其他未涵盖的杂项，枚举型，null表示没有，用'None'表示。
* MSZoning:房屋所处区域，枚举型，用最多的值填充。
* Utilities:可用公共设施(水电气等)类型，意义枚举型，用最多的值填充。
* Exterior1st:房屋外覆盖物，枚举型，用最多的值填充。
* Exterior2nd:额外的覆盖物，枚举型，用Exterior1st填充。
* KitchenQual:厨房质量，意义枚举型，null表示没有，用'None'表示。
* Functional:家庭功能，枚举型，用最多值填充。
* SaleType:销售方式，枚举型，用最多值填充。

### 3.22 Clean Data - full & drop

In [5]:
for dataset in data_cleaner:
    dataset['LotFrontage'].fillna(dataset['LotFrontage'].mean(), inplace=True)
    dataset['Alley'].fillna('None', inplace=True)
    dataset['MasVnrType'].fillna('None', inplace=True)
    dataset['MasVnrArea'].fillna(0, inplace=True)
    
    dataset['BsmtQual'].fillna('None', inplace=True)
    dataset['BsmtCond'].fillna('None', inplace=True)
    dataset['BsmtExposure'].fillna('None', inplace=True)
    dataset['BsmtFinType1'].fillna('None', inplace=True)
    dataset['BsmtFinSF1'].fillna(0, inplace=True)
    dataset['BsmtFinType2'].fillna('None', inplace=True)
    dataset['BsmtFinSF2'].fillna(0, inplace=True)
    dataset['BsmtUnfSF'].fillna(0, inplace=True)
    dataset['TotalBsmtSF'].fillna(0, inplace=True)
    dataset['BsmtFullBath'].fillna(0, inplace=True)
    dataset['BsmtHalfBath'].fillna(0, inplace=True)
    
    dataset['Electrical'].fillna(dataset['Electrical'].value_counts().index[0], inplace=True)
    dataset['FireplaceQu'].fillna('None', inplace=True)
    
    dataset['GarageType'].fillna('None', inplace=True)
    dataset['GarageYrBlt'].fillna(0, inplace=True)
    dataset['GarageFinish'].fillna('None', inplace=True)
    dataset['GarageQual'].fillna('None', inplace=True)
    dataset['GarageCond'].fillna('None', inplace=True)
    dataset['GarageCars'].fillna(0, inplace=True)
    dataset['GarageArea'].fillna(0, inplace=True)
    
    dataset['PoolQC'].fillna('None', inplace=True)
    dataset['Fence'].fillna('None', inplace=True)
    dataset['MiscFeature'].fillna('None', inplace=True)
    dataset['MSZoning'].fillna(dataset['MSZoning'].value_counts().index[0], inplace=True)
    dataset['Utilities'].fillna(dataset['Utilities'].value_counts().index[0], inplace=True)
    dataset['Exterior1st'].fillna(dataset['Utilities'].value_counts().index[0], inplace=True)
    dataset['Exterior2nd'].fillna(dataset['Exterior1st'], inplace=True)
    
    dataset['KitchenQual'].fillna('None', inplace=True)
    
    dataset['Functional'].fillna(dataset['Functional'].value_counts().index[0], inplace=True)
    
    dataset['SaleType'].fillna(dataset['SaleType'].value_counts().index[0], inplace=True)

In [6]:
for dataset in data_cleaner:
    dataset['age'] = 2012 - dataset['YearBuilt']
    dataset['SaleMonths'] = (2012-train_data['YrSold'])*12+train_data['MoSold']
    dataset['HasNotRemod'] = train_data['YearRemodAdd'] == train_data['YearBuilt']

In [7]:
test_id = test_data['Id'].copy()
remove_cols = ['Id','YearBuilt','YrSold','YearRemodAdd']
train_data.drop(remove_cols, axis=1, inplace=True)
test_data.drop(remove_cols, axis=1, inplace=True)

In [8]:
print train_data.info()
print '-'*100
print test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1460 non-null object
MasVnrArea       1460 

### 3.23 Convert Format

In [9]:
label = LabelEncoder()
for dataset in data_cleaner:
    for col in list(train_data.columns[train_data.dtypes == 'object']):
        dataset[col] = label.fit_transform(dataset[col])
#     dataset['MSZoning'] = label.fit_transform(dataset['MSZoning'])
#     dataset['Street'] = label.fit_transform(dataset['Street'])
#     dataset['Alley'] = label.fit_transform(dataset['Alley'])
#     dataset['LotShape'] = label.fit_transform(dataset['LotShape'])
#     dataset['LandContour'] = label.fit_transform(dataset['LandContour'])
#     dataset['Utilities'] = label.fit_transform(dataset['Utilities'])
#     dataset['LotConfig'] = label.fit_transform(dataset['LotConfig'])
#     dataset['LandSlope'] = label.fit_transform(dataset['LandSlope'])
#     dataset['Neighborhood'] = label.fit_transform(dataset['Neighborhood'])
#     dataset['Condition1'] = label.fit_transform(dataset['Condition1'])
#     dataset['Condition2'] = label.fit_transform(dataset['Condition2'])
#     dataset['BldgType'] = label.fit_transform(dataset['BldgType'])
#     dataset['HouseStyle'] = label.fit_transform(dataset['HouseStyle'])
#     dataset['RoofStyle'] = label.fit_transform(dataset['RoofStyle'])
#     dataset['Exterior1st'] = label.fit_transform(dataset['Exterior1st'])
    
#     dataset['FareBin_Code'] = label.fit_transform(dataset['Exterior2nd'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['MasVnrType'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['ExterQual'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['ExterCond'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['Foundation'])
    
#     dataset['FareBin_Code'] = label.fit_transform(dataset['BsmtQual'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['BsmtCond'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['BsmtExposure'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['BsmtFinType1'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['BsmtFinType2'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['Heating'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['HeatingQC'])
    
#     dataset['FareBin_Code'] = label.fit_transform(dataset['CentralAir'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['Electrical'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['KitchenQual'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['Functional'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['FireplaceQu'])
    
#     dataset['FareBin_Code'] = label.fit_transform(dataset['GarageType'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['GarageFinish'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['GarageQual'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['GarageCond'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['PavedDrive'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['PoolQC'])
    
#     dataset['FareBin_Code'] = label.fit_transform(dataset['Fence'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['MiscFeature'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['SaleType'])
#     dataset['FareBin_Code'] = label.fit_transform(dataset['SaleCondition'])

### 3.24 Double Check Cleaned Data

In [10]:
print'Train columns with null values: \n'+str(train_data.isnull().sum())
print("-"*100)
print train_data.info()
print("-"*100)

print 'Test columns with null values: \n'+str(test_data.isnull().sum())
print("-"*100)
print test_data.info()
print("-"*100)

Train columns with null values: 
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
                ..
KitchenQual      0
TotRmsAbvGrd     0
Functional       0
Fireplaces       0
FireplaceQu      0
GarageType       0
GarageYrBlt      0
GarageFinish     0
GarageCars       0
GarageArea       0
GarageQual       0
GarageCond       0
PavedDrive       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
PoolQC           

### 3.25 Split Data to Train and Test

In [11]:
train_target = train_data['SalePrice'].copy()
train_data.drop(['SalePrice'], axis=1, inplace=True)
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_data, train_target, random_state = 0)

## 3.3 Feature Selection

# Step 4:Perform Exloratory Analysis with Statistics

# Step 5:Model Data

## 5.1 Evaluate Model Performance

### 5.11 Model Performance with Cross-Validation (CV)

### 5.12 Tune Model with Hyper-Parameters

### 5.13 Tune Model with Feature Selection

# Step 6:Validate and Implement

# Step 7: Optimize and Strategize